In [ ]:
%pip install -e .
%pip install -r requirements.txt
%pip install nest_asyncio

In [2]:
import nest_asyncio

# Allow backtests to run in notebook
nest_asyncio.apply()

# Increase width of console output for browsers
from IPython.display import HTML, display

display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
import pandas as pd

from lazyft import backtest, paths, remote
from lazyft.backtest.runner import BacktestMultiRunner
from lazyft.config import Config
from lazyft.reports import get_backtest_repo, get_hyperopt_repo

2022-05-30 20:19:34.761 | WARNING  | lazyft.remote:<module>:25 - /home/raphael/PycharmProjects/lazyft_lib/remotes.json does not exist. Remotes not loaded


# Prepare to backtest

In [3]:
from lazyft.command_parameters import BacktestParameters

b_timerange = "20220301-20220401"
# b_days = 30

b_conf = Config('config.json')

b_params = BacktestParameters(
    timerange=b_timerange,
    interval='1h',
    config_path=b_conf,
    # days=b_days,
    # secrets_config=priv_bin_us,
    # pairs=['PYR/USDT'],
    stake_amount='unlimited',
    # timeframe_detail='5m',
    starting_balance=100,
    max_open_trades=3,
    download_data=True,
    tag='',
)
print(len(b_params.pairs), 'pairs')
# b_params.max_open_trades = len(b_params.pairs)
# print('strategy:', b_params.strategies[0])
print('max open trades:', b_params.max_open_trades)
print('stake amount:', b_params.stake_amount)
print('timeframe:', b_params.timerange)
print('starting balance:', b_params.starting_balance)
print('exchange:', b_params.config['exchange']['name'])
print('timeframe:', b_params.interval)
if b_params.ensemble:
    print('ensemble:', b_params.ensemble)

29 pairs
max open trades: 3
stake amount: unlimited
timeframe: 20220301-20220401
starting balance: 100
exchange: binance
timeframe: 1h


## Run Backtest on Gumbo1

In [7]:
b_runner = b_params.run('Gumbo1', load_from_hash=False)
b_runner.save()

2022-05-30 20:48:22.326 | INFO     | lazyft.parameter_tools:remove_params_file:36 - Removing strategy params: /home/raphael/PycharmProjects/lazyft_lib/user_data/strategies/Gumbo1.json
2022-05-30 20:48:22.340 | INFO     | lazyft.strategy:save_strategy_text_to_database:201 - Strategy Gumbo1 already in database...skipping
2022-05-30 20:48:23.212 | INFO     | lazyft.downloader:download_missing_historical_data:315 - Checking if download is needed for ETH/USDT, TRX/USDT, GALA/USDT, EUR/USDT, ETC/USDT, RAD/USDT, APE/USDT, SOL/USDT, BTC/USDT, SAND/USDT, GMT/USDT, LRC/USDT, MATIC/USDT, BURGER/USDT, MANA/USDT, BUSD/USDT, XMR/USDT, FTM/USDT, YFII/USDT, DOGE/USDT, XRP/USDT, USDC/USDT, CHESS/USDT, SHIB/USDT, ROSE/USDT, DOT/USDT, NEAR/USDT, AVAX/USDT, ADA/USDT, BTC/USDT, ETH/USDT, BUSD/USDT, USDC/USDT, XRP/USDT, SOL/USDT, GMT/USDT, TRX/USDT, SAND/USDT, AVAX/USDT, FTM/USDT, APE/USDT, NEAR/USDT, DOT/USDT, ADA/USDT, YFII/USDT, MANA/USDT, GALA/USDT, SHIB/USDT, LRC/USDT, RAD/USDT, MATIC/USDT, BURGER/USDT

BacktestReport(date=datetime.datetime(2022, 5, 30, 20, 48, 25, 506573), session_id=None, tag='20220301-20220401', hyperopt_id='', strategy_hash='59eb4b984906f52dd73aaa7da1031e3c', id=1, hash='ae33447b44fd8a3f4eea6939c9937d73', ensemble='', backtest_file_str='backtest-result-2022-05-30_20-48-25.json', exchange='binance')

## Show backtest

In [8]:
print(b_runner.report.report_text)

Result for strategy Gumbo1 #1 | Hyperopt: False | Hyperopt id: 
=========================================================== BACKTESTING REPORT ===========================================================
|        Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|-------------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|    FTM/USDT |      4 |           1.09 |           4.38 |             1.472 |           1.47 |       10:30:00 |     2     2     0   100 |
|    GMT/USDT |      2 |           1.85 |           3.70 |             1.259 |           1.26 |       10:00:00 |     1     1     0   100 |
|    RAD/USDT |      1 |           2.23 |           2.23 |             0.741 |           0.74 |        7:00:00 |     1     0     0   100 |
|   GALA/USDT |      1 |           2.17 |           2.17 |             0.718 |           0.72 |       

In [18]:
b_runner.report.df

,id,strategy,hyperopt_id,date,exchange,m_o_t,stake,balance,n_pairlist,avg_profit_pct,avg_duration,wins,losses,sortino,drawdown,total_profit_pct,total_profit,trades,days,tag
0,None,Gumbo1,,05/30/22 19:59:33,binance,3,unlimited,100,30,-0.020,20:35:00,6,2,-3.692,0.111,-0.078,-7.810,12,31,20220301-20220401


In [21]:
b_runner.report.plot_weekly()

# Prepare to Hyperopt Gumbo1

In [29]:
from lazyft.command_parameters import HyperoptParameters

h_days = 365
h_conf = Config('config.json')
starting_balance = 100
h_timerange = "20220101-20220331"
# VRA/USDT HTR/USDT ARX/USDT
params = HyperoptParameters(
    epochs=50,
    config_path=h_conf,
    #     secrets_config=str(priv_us),
    # days=h_days,
    timerange=h_timerange,
    spaces="default",
    # loss='CalmarHyperOptLoss',
    loss='SortinoHyperOptLoss',
    interval="1h",
    # min_trades=int(days * 0.67 * 0.85),
    min_trades=10,
    #     pairs=pairlist.load_pairlist_from_id(1),
    starting_balance=starting_balance,
    max_open_trades=3,
    stake_amount="unlimited",
    jobs=-2,
    download_data=True,
    # custom_spaces='atr_roi atr_stoploss',
    # custom_settings={
    #     'use_custom_stoploss': False,
    # },
    tag="",
    # seed=62581,
    #     ensemble=ensemble
)
# params.tag = params.tag + ',' + params.pairs[0]
# params.min_trades = len(params.pairs)
print("\ntimerange: ", params.timerange,
      "\ndays: ", round(params.days * 0.667),
      '\nspaces:', params.spaces,
      '\nloss function:', params.loss,
      '\nepochs:', params.epochs,
      '\nmin trades:', params.min_trades,
      '\ntag:', params.tag,
      '\ninterval:', params.interval,
      '\nnum pairs:', len(params.pairs))
# QuickTools.refresh_pairlist(us_conf, 100)


timerange:  20220101-20220331 
days:  40 
spaces: default 
loss function: SortinoHyperOptLoss 
epochs: 50 
min trades: 10 
tag: 20220101-20220331,default 
interval: 1h 
num pairs: 30


## Run Hyperopt

In [30]:
h_runner = params.run('Gumbo1', background=True)

2022-05-30 20:08:16.298 | INFO     | lazyft.hyperopt.runner:pre_execute:185 - Preparing to hyperopt Gumbo1
2022-05-30 20:08:17.296 | INFO     | lazyft.downloader:download_missing_historical_data:315 - Checking if download is needed for BTC/USDT, YFII/USDT, XRP/USDT, AVAX/USDT, MANA/USDT, NEAR/USDT, CHESS/USDT, GMT/USDT, BUSD/USDT, ETC/USDT, DOT/USDT, SHIB/USDT, BURGER/USDT, DOGE/USDT, USDC/USDT, SAND/USDT, FTM/USDT, RAD/USDT, LRC/USDT, APE/USDT, GALA/USDT, XMR/USDT, ANC/USDT, EUR/USDT, ROSE/USDT, TRX/USDT, SOL/USDT, ADA/USDT, MATIC/USDT, ETH/USDT, BTC/USDT, ETH/USDT, BUSD/USDT, USDC/USDT, XRP/USDT, SOL/USDT, ANC/USDT, GMT/USDT, TRX/USDT, SAND/USDT, AVAX/USDT, FTM/USDT, APE/USDT, NEAR/USDT, DOT/USDT, ADA/USDT, YFII/USDT, MANA/USDT, GALA/USDT, SHIB/USDT, LRC/USDT, RAD/USDT, MATIC/USDT, BURGER/USDT, ETC/USDT, XMR/USDT, ROSE/USDT, DOGE/USDT, EUR/USDT, CHESS/USDT @ 1h interval(s)
2022-05-30 20:08:17.380 | INFO     | lazyft.downloader:download_missing_historical_data:360 - Data is up to date

+--------+---------+----------+------------------+--------------+-------------------------------+-----------------+-------------+-------------------------------+

|   Best |   Epoch |   Trades |    Win Draw Loss |   Avg profit |                        Profit |    Avg duration |   Objective |           Max Drawdown (Acct) |

|--------+---------+----------+------------------+--------------+-------------------------------+-----------------+-------------+-------------------------------|

| * Best |    1/50 |       31 |     22    2    7 |        0.17% |         0.285 USDT    (0.28%) | 0 days 14:17:00 |    -0.19016 |        20.442 USDT   (20.44%) |

...

trailing_stop = False  # value loaded from strategy

trailing_stop_positive = None  # value loaded from strategy

trailing_stop_positive_offset = 0.0  # value loaded from strategy

trailing_only_offset_is_reached = False  # value loaded from strategy

2022-05-30 20:09:36.819 | INFO     | lazyft.parameter_tools:remove_params_file:36 - Removing strategy params: /home/raphael/PycharmProjects/lazyft_lib/user_data/strategies/Gumbo1.json
2022-05-30 20:09:36.820 | SUCCESS  | lazyft.hyperopt.runner:on_finished:300 - Finished successfully.
2022-05-30 20:09:36.888 | INFO     | lazyft.strategy:delete_temporary_strategy_backup_dir:262 - Temporary folder "None" does not exist...skipping


## Show

In [32]:
h_runner.report.df

,id,strategy,date,exchange,m_o_t,stake,balance,n_pairlist,avg_profit_pct,avg_duration,wins,losses,drawdown,total_profit_pct,total_profit,trades,days,tag
0,None,Gumbo1,05/30/22 20:09:36,binance,3,unlimited,100,30,0.004,17:59:00,40,13,0.204,0.071,7.070,66,80,"20220101-20220331,default"


## Save

In [33]:
h_runner.save()

2022-05-30 20:10:18.217 | INFO     | lazyft.hyperopt.runner:save:339 - Created report id: 1


HyperoptReport(id=1, epoch=41, strategy_hash='59eb4b984906f52dd73aaa7da1031e3c', hyperopt_file_str='/home/raphael/PycharmProjects/lazyft_lib/user_data/hyperopt_results/strategy_Gumbo1_2022-05-30_20-08-22.fthypt', date=datetime.datetime(2022, 5, 30, 20, 9, 36, 821268), tag='20220101-20220331,default', exchange='binance')

# Backtest Gumbo1 Using Hyperopted Parameters

In [9]:
bh_runner = b_params.run('Gumbo1-1', load_from_hash=False)

2022-05-30 20:48:48.304 | INFO     | lazyft.models:export_parameters:438 - Exported parameters for report 1 to /home/raphael/PycharmProjects/lazyft_lib/user_data/strategies/Gumbo1.json
2022-05-30 20:48:48.306 | INFO     | lazyft.strategy:create_temp_folder_for_strategy_and_params_from_backup:228 - Created temporary folder /tmp/lazyft-Gumbo1_gxy3c7qa for strategy backup Gumbo1
2022-05-30 20:48:48.318 | INFO     | lazyft.models:export_to:1019 - Exported strategy Gumbo1 with hash 59eb4b984906f52dd73aaa7da1031e3c to /tmp/lazyft-Gumbo1_gxy3c7qa/Gumbo1.py
2022-05-30 20:48:48.319 | INFO     | lazyft.strategy:create_temp_folder_for_strategy_and_params_from_backup:232 - Exported strategy backup Gumbo1 to /tmp/lazyft-Gumbo1_gxy3c7qa/Gumbo1.py
2022-05-30 20:48:48.322 | INFO     | lazyft.models:export_parameters:438 - Exported parameters for report 1 to /tmp/lazyft-Gumbo1_gxy3c7qa/Gumbo1.json
2022-05-30 20:48:48.323 | INFO     | lazyft.strategy:create_temp_folder_for_strategy_and_params_from_backu

In [10]:
bh_runner.save()

2022-05-30 20:48:58.933 | INFO     | lazyft.backtest.runner:save:360 - Created report id 2: {'start_date': datetime.datetime(2022, 3, 1, 0, 0), 'end_date': datetime.datetime(2022, 4, 1, 0, 0), 'trades': 13, 'wins': 9, 'losses': 1, 'draws': 3, 'drawdown': 0.00042888225715765444, 'avg_duration': '15:14:00', 'profit_mean_pct': 0.03359514153846154, 'profit_sum_pct': 43.67, 'profit_total_abs': 15.30402821, 'profit_total_pct': 0.153, 'duration_avg': datetime.timedelta(seconds=54840)}


BacktestReport(date=datetime.datetime(2022, 5, 30, 20, 48, 51, 880082), session_id=None, tag='20220301-20220401', hyperopt_id='1', strategy_hash='59eb4b984906f52dd73aaa7da1031e3c', id=2, hash='bc0be82b5836ab501643dbb5a1c17b83', ensemble='', backtest_file_str='backtest-result-2022-05-30_20-48-51.json', exchange='binance')

## Show

In [5]:
print(bh_runner.report.report_text)

Result for strategy Gumbo1 #None | Hyperopt: False | Hyperopt id: 1
=========================================================== BACKTESTING REPORT ===========================================================
|        Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|-------------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
|    GMT/USDT |      2 |           5.74 |          11.49 |             4.252 |           4.25 |       13:30:00 |     2     0     0   100 |
|    FTM/USDT |      5 |           2.30 |          11.49 |             4.074 |           4.07 |       17:36:00 |     2     3     0   100 |
| BURGER/USDT |      2 |           5.74 |          11.49 |             3.934 |           3.93 |       13:00:00 |     2     0     0   100 |
|   GALA/USDT |      1 |           4.56 |           4.56 |             1.507 |           1.51 |   

In [6]:
bh_runner.report.plot() 

# Browse Repo

In [11]:
get_backtest_repo().df()

,strategy,hyperopt_id,date,exchange,m_o_t,stake,balance,n_pairlist,avg_profit_pct,avg_duration,wins,losses,sortino,drawdown,total_profit_pct,total_profit,trades,days,tag
id,,,,,,,,,,,,,,,,,,,
2,Gumbo1,1,05/30/22 20:48:51,binance,3,unlimited,100,29,3.360,15:14:00,9,1,832.213,0.000,0.153,15.300,13,31,20220301-20220401
1,Gumbo1,,05/30/22 20:48:25,binance,3,unlimited,100,29,1.301,8:12:00,6,0,1.04K,0.000,0.044,4.370,10,31,20220301-20220401


In [13]:
get_backtest_repo().get(1)

BacktestReport(date=datetime.datetime(2022, 5, 30, 20, 48, 25, 506573), session_id=None, tag='20220301-20220401', hyperopt_id='', strategy_hash='59eb4b984906f52dd73aaa7da1031e3c', id=1, hash='ae33447b44fd8a3f4eea6939c9937d73', ensemble='', backtest_file_str='backtest-result-2022-05-30_20-48-25.json', exchange='binance')

In [12]:
get_hyperopt_repo().df()

,strategy,date,exchange,m_o_t,stake,balance,n_pairlist,avg_profit_pct,avg_duration,wins,losses,drawdown,total_profit_pct,total_profit,trades,days,tag
id,,,,,,,,,,,,,,,,,
1,Gumbo1,05/30/22 20:09:36,binance,3,unlimited,100,30,0.437,17:59:00,40,13,0.204,0.071,7.070,66,80,"20220101-20220331,default"
